# CS145: Project 1 | Me, in Texts

## Author
* *Kabir Jolly, kjolly*

## Project Overview

---

This project explores my digital footprint, attempting to better uncover who I am and what my relationships look like based on the largest digital repository of data I have on myself, my text messages.

The main question I am hoping to answer is:

*   Has the nature of the communication I have with my online relationships changed over the past decade (ages 12 till 22)?


 Some supporting questions that come up in the event to uncover the central answer include:

*   Who are the top contacts I've communicated with throughout time?
*   Has my communication style changed over the years?
*   What individual and group communications drive the bulk of my online interactions?





---


## Analysis of Dataset

---



Dataset overview and explanation:
  * File size: 1.09GB (original .db file)
  * Relevant tables: mainTable (constructed via INSERT query from the below)
    * `message` Relevant text message information with ROWID primary key (PK `rowid` and FK `handle_id`-> `handle.rowid`)
    * `chat_message_join` Join table between `message` and `chat` (FK `message_id`-> `message.row_id` and `chat_id`->`chat.rowid)
    * `chat_handle_join` Join table between chats and handles (chat participants) (FK `chat_id`-> `chat.row_id` and `chat_id`->`chat.rowid)
    * `handle` Represents contacts and identifies group chats or DMs (FK `message.handle_id`-> `chat_handle_join.handle_id`)
    * `chat` Contains metadata about DM and group chats

Data issues:
*   There was a lot of empty message data which I realized was not simply found in the raw string entry. I found a script online that takes it from the AttributedBody column and fills in the missing text (formatted as NSAttributedBody, a MacOS typing that needs to be parsed). This step nearly doubled the amount of data I had at my disposal for the project. Credit for this quick fix is found [here](https://apple.stackexchange.com/questions/421665/how-specificially-do-i-read-a-chat-db-file).
*   For uploading to big query, I decided to turn the .db file into a Parquet to easily read it without the issues of using a delimiter-based file format (as that was creating inconsistencies in the file) and so I ran JOINs and SELECTs to create my working dataset window (1.09GB -> 107MB of relevant data). The code I used is as follows:


```
INSERT INTO mainTable (rowid, ThreadId, IsFromMe, FromPhoneNumber, ToPhoneNumber, Service, TextDate, MessageText, AttributedBody, RoomName, ContactName, AssociatedMessageType, GUID, AssociatedMessageGUID)
  SELECT
      m.rowid,
      COALESCE(m.cache_roomnames, h.id) AS ThreadId,
      m.is_from_me AS IsFromMe,
      CASE
          WHEN m.is_from_me = 1 THEN 'Me'
          ELSE h.id
      END AS FromPhoneNumber,
      CASE
          WHEN m.is_from_me = 0 THEN 'Me'
          ELSE COALESCE(h2.id, h.id)
      END AS ToPhoneNumber,
      m.service AS Service,
      DATETIME((m.date / 1000000000) + 978307200, 'unixepoch', 'localtime') AS TextDate,
      m.text AS MessageText,
      m.attributedBody AS AttributedBody,
      c.display_name AS RoomName,
      NULL AS ContactName,
      m.associated_message_type AS AssociatedMessageType,
      m.guid AS GUID,
      m.associated_message_guid AS AssociatedMessageGUID
  FROM
      message AS m
      LEFT JOIN chat_message_join AS cmj ON m.rowid = cmj.message_id
      LEFT JOIN chat_handle_join AS chj ON cmj.chat_id = chj.chat_id
      LEFT JOIN handle AS h ON
          (m.cache_roomnames IS NOT NULL AND h.rowid = m.handle_id) OR
          (m.cache_roomnames IS NULL AND h.rowid = chj.handle_id)
      LEFT JOIN chat AS c ON m.cache_roomnames = c.room_name
      LEFT JOIN handle AS h2 ON chj.handle_id = h2.rowid
  WHERE NOT EXISTS (
      SELECT 1 FROM mainTable WHERE mainTable.rowid = m.rowid
  )
  GROUP BY m.rowid
  ORDER BY
      m.date DESC;
```




----

## Data Exploration

---

*Explore the dataset, my central questions, and supplementary questions*

---

In [1]:
# Run this cell to authenticate yourself to BigQuery
from google.colab import auth
auth.authenticate_user() # make sure to "Select All" when authenticating
project_id = "cs145-project1-receipts"

In [2]:
# Initialize BiqQuery client
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

In [3]:
%%bigquery --project $project_id

-- Total number of messages I sent or received (excluding things like reactions)

SELECT
  -- Add together messages sent/received NOT in group chats
  SUM(CASE
      WHEN IsFromMe = 0 AND RoomName IS NULL THEN 1
      ELSE 0
  END
    ) AS ReceivedMessages,
  SUM(CASE
      WHEN IsFromMe = 1 AND RoomName IS NULL THEN 1
      ELSE 0
  END
    ) AS SentMessages
FROM
  `cs145-project1-receipts.texts_dataset.texts`
-- Account for reactions/media/apple pay/etc
WHERE
  AssociatedMessageType = 0;

Query is running:   0%|          |

Downloading:   0%|          |

,ReceivedMessages,SentMessages
0,162901,139190


### 4a) SQL queries exploring key questions with proper structure and helpful comments

In [4]:
%%bigquery --project $project_id

-- Aggregate number of messages sent and received year over year

SELECT
  -- add year based on timestamp parsing
  FORMAT_TIMESTAMP('%Y', PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', TextDate)) AS Year,
  COUNT(*) AS MessageCount
FROM
  `cs145-project1-receipts.texts_dataset.texts`
-- aggregate count by year
GROUP BY
  Year
ORDER BY
  Year;

Query is running:   0%|          |

Downloading:   0%|          |

,Year,MessageCount
0,2013,819
1,2014,3095
2,2015,12682
3,2016,18661
4,2017,55683
5,2018,50146
6,2019,39295
7,2020,24038
8,2021,51415
9,2022,85510


In [5]:
%%bigquery --project $project_id

-- average message length over the years

SELECT
  FORMAT_TIMESTAMP('%Y', PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', TextDate)) AS Year,
  AVG(LENGTH(MessageText)) AS AvgMessageLength
FROM
  `cs145-project1-receipts.texts_dataset.texts`
WHERE
  MessageText IS NOT NULL
GROUP BY
  Year
ORDER BY
  Year;

Query is running:   0%|          |

Downloading:   0%|          |

,Year,AvgMessageLength
0,2013,17.409035
1,2014,41.296607
2,2015,30.262971
3,2016,24.129254
4,2017,31.839628
5,2018,41.407849
6,2019,40.706757
7,2020,43.016973
8,2021,41.202762
9,2022,38.258636


In [6]:
%%bigquery --project $project_id

-- 10 most DIRECTLY messaged individuals (based on sum of sent/received)

-- Create table of non-groupchat text messages attached to a contact name
WITH FilteredMessages AS (
  SELECT
    ContactName,
    IsFromMe
  FROM
    `cs145-project1-receipts.texts_dataset.texts`
  WHERE
    ContactName IS NOT NULL
    AND RoomName IS NULL
    AND AssociatedMessageType = 0
)
SELECT
  ContactName,
  COUNT(*) AS TotalMessages,
  SUM(CASE WHEN IsFromMe = 1 THEN 1 ELSE 0 END) AS SentMessages,
  SUM(CASE WHEN IsFromMe = 0 THEN 1 ELSE 0 END) AS ReceivedMessages
FROM
  FilteredMessages
GROUP BY
  ContactName
ORDER BY
  TotalMessages DESC
-- Top 10 from highest sums (similar to earlier query)
LIMIT
  10;


Query is running:   0%|          |

Downloading:   0%|          |

,ContactName,TotalMessages,SentMessages,ReceivedMessages
0,Yubin Jee,19585,9698,9887
1,Michael Gerhard,18272,8601,9671
2,Akshay Gupta,13732,6787,6945
3,Jacob Faierman,12535,6124,6411
4,Ronin Burke,10796,5061,5735
5,Aa Mom,9876,4450,5426
6,Daniel Fein,9616,4768,4848
7,Emily Hui,8610,3290,5320
8,Jeremiah Elizabe,7878,2988,4890
9,Matt Kaplan,7868,3640,4228


### 4b) SQL queries to answer supplementary questions through meaningful data exploration

In [7]:
%%bigquery --project $project_id

-- Top 20 most used group chats I'm in

WITH GroupChatRankings AS (
  SELECT
    RoomName,
    COUNT(*) as NumMessagesInGroupChat
  FROM
    `cs145-project1-receipts.texts_dataset.texts`
  WHERE
    RoomName IS NOT NULL
  GROUP BY
    RoomName
),
RankedGroupChats AS (
  SELECT
    RoomName,
    NumMessagesInGroupChat,
    RANK() over (ORDER BY NumMessagesInGroupChat DESC) ChatRank
  FROM
    GroupChatRankings
)
  SELECT
    ChatRank,
    RoomName,
    NumMessagesInGroupChat
  FROM
    RankedGroupChats
  ORDER BY
    ChatRank ASC
  LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,ChatRank,RoomName,NumMessagesInGroupChat
0,1,🇨🇳 🇫🇷 🇵🇹 🇲🇽 🇮🇳,67921
1,2,,57327
2,3,Old Freshmen Flags Chat,16834
3,4,Margaritaville,13933
4,5,The Jollys,11276
5,6,Built like a fat 👵,4372
6,7,MMM,4363
7,8,gambling buddies,4197
8,9,fantasy football,4059
9,10,cs one-forty-sending-it,3806


In [8]:
%%bigquery --project $project_id

-- Number of conversations and average length of messages in those, giving me activity over time

WITH YearlyMessages AS (
  -- Extract year and messages/conversation
  SELECT
    FORMAT_TIMESTAMP('%Y', PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', TextDate)) AS Year,
    ThreadId,
    COUNT(*) AS MessageCount
  FROM
    `cs145-project1-receipts.texts_dataset.texts`
  WHERE
    MessageText IS NOT NULL
  GROUP BY
    Year, ThreadId
),
ConversationAnalysis AS (
  -- Distinct conversations/year and avg message count
  SELECT
    Year,
    COUNT(DISTINCT ThreadId) AS ConversationCount,
    AVG(MessageCount) AS AvgMessagesPerConversation
  FROM
    YearlyMessages
  GROUP BY
    Year
)
SELECT
  Year,
  ConversationCount,
  AvgMessagesPerConversation
FROM
  ConversationAnalysis
ORDER BY
  Year;

Query is running:   0%|          |

Downloading:   0%|          |

,Year,ConversationCount,AvgMessagesPerConversation
0,2013,19,43.105263
1,2014,69,44.855072
2,2015,131,96.075758
3,2016,152,121.967320
4,2017,178,311.078212
5,2018,242,206.362140
6,2019,206,189.830918
7,2020,164,145.684848
8,2021,291,176.078767
9,2022,617,138.365696


In [9]:
%%bigquery --project $project_id

-- Over all my texts, find trends in texting frequency throughout the day

SELECT
  FORMAT_TIMESTAMP('%H', PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', TextDate)) AS Hour,
  COUNT(*) AS MessageCount
FROM
  `cs145-project1-receipts.texts_dataset.texts`
WHERE
  ContactName IS NOT NULL
  AND RoomName IS NULL
  AND AssociatedMessageType = 0
GROUP BY
  Hour
ORDER BY
  Hour;

Query is running:   0%|          |

Downloading:   0%|          |

,Hour,MessageCount
0,00,12108
1,01,9160
2,02,7202
3,03,4571
4,04,2416
5,05,1303
6,06,1246
7,07,2898
8,08,4229
9,09,4684


### 4c) SQL queries for trends, anomalies, or significant features in your data

In [12]:
%%bigquery --project $project_id

-- Are there discoverable trends in iMessage vs Android/Other(SMS) usage in my text history

SELECT
  FORMAT_TIMESTAMP('%Y', PARSE_TIMESTAMP('%Y-%m-%d %H:%M:%S', TextDate)) AS Year,
  SUM(CASE WHEN Service = 'iMessage' THEN 1 ELSE 0 END) AS iMessage,
  SUM(CASE WHEN Service = 'SMS' THEN 1 ELSE 0 END) AS SMS,
FROM
  `cs145-project1-receipts.texts_dataset.texts`
GROUP BY
  Year
ORDER BY
  Year;

Query is running:   0%|          |

Downloading:   0%|          |

,Year,iMessage,SMS
0,2013,166,653
1,2014,2558,537
2,2015,11336,1346
3,2016,16380,2281
4,2017,42411,13272
5,2018,37992,12154
6,2019,30211,9084
7,2020,21297,2741
8,2021,41333,10082
9,2022,79965,5544


In [10]:
%%bigquery --project $project_id

-- Compare the reactions (significant dataset feature) I send (IsFromMe=1) versus the reactions I receive (IsFromMe=0)

-- Create Reactions table containing only reaction data
WITH
  Reactions AS (
  SELECT
    IsFromMe,
    AssociatedMessageType
  FROM
    `cs145-project1-receipts.texts_dataset.texts`
  WHERE
    AssociatedMessageType = 2000    -- love
    OR AssociatedMessageType = 2001 -- like
    OR AssociatedMessageType = 2002 -- dislike
    OR AssociatedMessageType = 2003 -- laugh
    OR AssociatedMessageType = 2004 -- emphasize
    OR AssociatedMessageType = 2005 -- question
  )
SELECT
  IsFromMe,
  AssociatedMessageType,
  COUNT(*) AS MessageCount
FROM
  Reactions
GROUP BY
  IsFromMe,
  AssociatedMessageType
ORDER BY
  AssociatedMessageType,
  IsFromMe;

Query is running:   0%|          |

Downloading:   0%|          |

,IsFromMe,AssociatedMessageType,MessageCount
0,0,2000,9915
1,1,2000,5558
2,0,2001,5801
3,1,2001,3309
4,0,2002,589
5,1,2002,163
6,0,2003,3968
7,1,2003,3685
8,0,2004,4426
9,1,2004,1903


In [11]:
%%bigquery --project $project_id

-- Showcase the difference in messages sent/received vs words sent/received for outlier
-- A close friend who was amongst my top texted had a massive disparity between messages sent/received
-- However, was she texting more, or can we explain this with measuring words rather than messages?

SELECT
    ContactName,
    SUM(CASE WHEN IsFromMe = 0 AND RoomName IS NULL THEN LENGTH(MessageText) - LENGTH(REPLACE(MessageText, ' ', '')) + 1 ELSE 0 END) AS ReceivedWords,
    SUM(CASE WHEN IsFromMe = 1 AND RoomName IS NULL THEN LENGTH(MessageText) - LENGTH(REPLACE(MessageText, ' ', '')) + 1 ELSE 0 END) AS SentWords,
    SUM(CASE WHEN IsFromMe = 0 THEN 1 ELSE 0 END) AS ReceivedMessages,
    SUM(CASE WHEN IsFromMe = 1 THEN 1 ELSE 0 END) AS SentMessages
FROM
    `cs145-project1-receipts.texts_dataset.texts`
WHERE
    ContactName = "Emily Hui"
    AND RoomName IS NULL
    AND AssociatedMessageType = 0
    AND MessageText IS NOT NULL
GROUP BY
    ContactName
ORDER BY
    ContactName;

Query is running:   0%|          |

Downloading:   0%|          |

,ContactName,ReceivedWords,SentWords,ReceivedMessages,SentMessages
0,Emily Hui,25997,25466,5320,3290


## Conclusion

---

Final conclusions based on the rest of your project

---

This project taught me a great deal beyond solidifying my skills in writing some fairly intricate SQL commands. From the data, I uncovered a variety of trends, such as that I text more as the years go on, except for in 2017 where as a freshman in high school I was really texting a lot. It showed me who my most contacted individuals where, uncovering insights about how college friends have quickly taken the podium on close friends I have known my whole life. I found that I react to messages a lot, loving and laughing at messages more than I ever thought. In conclusion, I realized that my habits have changed significantly and my digital communication mirrors much of the changes I view in my 'IRL' life. There are a lot of patterns and trends that were uncovered, with some interesting ones being gender differences in texting habits (quantity of messages vs quantity of words) held to be true amonst several examples I tried within my most texted people. It also mirrored my relationships that operate mostly in a group versus individual setting and how much my overall communication patterns have shifted in the 10+ years of 500,000+ messages I have sent.

Beyond that, I think it is important to note some limitations. You cannot capture semantic nuance in a declarative-friendly SQL paradigm. Things like sentiment, behavior, and and the differences between spoken and typed language are missing from these analyses, despite being crucial to painting the picture on such a data repository.

In the future, I would like to have more complex queries regarding response time of individuals, extract more insights like emoji usage, or breakdown groupchats and DM conversations into granular insights (rather than the focus of this project which ended up being holistic insights across the entire dataset). Beyond database query construction, there is much space for writing ML/DL models for data analysis to pull out non-linear insights and leverage LLM technologies to report findings that are harder to uncover. You could simulate conversations, learn how to communicate better, or figure out where your conversational relationships are going right or wrong.